In [17]:
import requests as r
import time
from datetime import datetime
from datetime import timedelta, time
import pandas as pd

'''
All HN API requests are constructed using the following reference: https://github.com/HackerNews/API
'''

base_url = "https://hacker-news.firebaseio.com/v0/"
human_display_item_url = "https://news.ycombinator.com/item?id="
new_stories_slug = "newstories"
item_slug = "item/"
json_append_slug = ".json"

In [2]:
### Get Story IDs for the newest stories from the HN New Stories endpoint
new_stories_response = r.get(base_url + new_stories_slug + json_append_slug)
new_stories_json = new_stories_response.json()

In [4]:
# returns datetime.timedelta object representing time between execution and story posting
def get_td_from_story(story):
    story_unix_time = story['time']
    story_datetime = datetime.fromtimestamp(story_unix_time)
    timedelta_since_post = datetime.now() - story_datetime
    return timedelta_since_post

def get_comments_per_minute_from_story(timedelta_since_post, num_comments):
    minutes_since_post = timedelta_since_post.seconds / 60
    comments_per_minute = (num_comments * 1.0) / minutes_since_post
    return comments_per_minute

def format_timedelta(timedelta_since_post):
    hours_since_post, remainder = divmod(timedelta_since_post.seconds, 3600)
    minutes_since_post, seconds = divmod(remainder, 60)
    time_since_post = str(hours_since_post) + 'h' + str(minutes_since_post) + 'm'
    return time_since_post

def get_comment_timestamps(story_json, delay_in_seconds):
    comments = story_json['kids']
    comment_timestamps = []
    for comment_id in comments:
        comment_response = r.get(base_url + item_slug + str(comment_id) + json_append_slug)
        comment = comment_response.json()
        comment_timestamps.append(comment['time'])
        time.sleep(delay_in_seconds)
    return comment_timestamps

def get_item_timestamps_for_all_kids(item_json, delay_in_seconds):
    if 'kids' not in item_json:
        return []
    kids = item_json['kids']
    print("Pulling timestamp info of kids for item with " + str(len(kids)) + " kids")
    kid_timestamps = []
    for kid_id in kids:
        kid_response = r.get(base_url + item_slug + str(kid_id) + json_append_slug)
        kid_json = kid_response.json()
        kid_timestamps.append(kid_json['time'])
        kid_timestamps = kid_timestamps + get_item_timestamps_for_all_kids(kid_json, delay_in_seconds)
        time.sleep(delay_in_seconds)
    return kid_timestamps

# Structure: story_id: {num_comments, }
story_info_tracker = {}
num_stories = len(new_stories_json)
comment_timeseries_records = []
for i, story_id in enumerate(new_stories_json):
    story_response = r.get(base_url + item_slug + str(story_id) + json_append_slug)
    story = story_response.json()
    if 'descendants' in story:
        num_comments = story['descendants']
    elif ('dead' in story and story['dead']):
        continue
    elif ('deleted' in story and story['deleted']):
        continue
    else:
        raise Exception("Couldn't find descendants in story: " + str(story))
    title = story['title']
    timedelta_since_post = get_td_from_story(story)
    comments_per_minute = get_comments_per_minute_from_story(timedelta_since_post, num_comments)
    time_since_post = format_timedelta(timedelta_since_post)
    if num_comments > 0:
        item_timestamps = get_item_timestamps_for_all_kids(story, 0.1)
    else:
        item_timestamps = []
    for timestamp in item_timestamps:
        comment_timeseries_records.append({'story_id': story_id, 
                                           'story_title': title, 
                                           'comment_timestamp': datetime.fromtimestamp(timestamp)})
    story_info_tracker[story_id] = {
        'num_comments': num_comments,
        'title': title,
        'timedelta_since_post': timedelta_since_post,
        'time_since_post': time_since_post,
        'comments_per_minute': comments_per_minute,
        'url': human_display_item_url + str(story_id),
        'comment_timestamps': item_timestamps,
    }
    print("Pulled info for story " + str(i) + "/" + str(num_stories))

Pulled info for story 0/500
Pulled info for story 1/500
Pulled info for story 2/500
Pulled info for story 3/500
Pulling timestamp info of kids for item with 2 kids
Pulled info for story 4/500
Pulled info for story 5/500
Pulled info for story 6/500
Pulled info for story 7/500
Pulled info for story 8/500
Pulled info for story 9/500
Pulled info for story 10/500
Pulled info for story 11/500
Pulled info for story 12/500
Pulling timestamp info of kids for item with 1 kids
Pulled info for story 13/500
Pulled info for story 14/500
Pulled info for story 15/500
Pulling timestamp info of kids for item with 1 kids
Pulled info for story 16/500
Pulled info for story 17/500
Pulled info for story 18/500
Pulled info for story 19/500
Pulling timestamp info of kids for item with 2 kids
Pulled info for story 20/500
Pulled info for story 21/500
Pulled info for story 22/500
Pulled info for story 23/500
Pulled info for story 24/500
Pulled info for story 25/500
Pulled info for story 26/500
Pulled info for sto

In [7]:
comment_timeseries_df = pd.DataFrame.from_records(comment_timeseries_records)

comment_timeseries_df.to_csv('comment_timeseries_df.csv')

In [8]:
# Gets comments per minute from an array of unix timestamps representing comment times, and minutes in the past to consider recent.
def get_comments_per_minute_from_recent_comments(comment_unix_timestamp_array, minutes_to_consider_recent):
    recency_datetime = datetime.now() - timedelta(minutes=minutes_to_consider_recent)
    filtered_comments = [comment for comment in comment_unix_timestamp_array if comment >= recency_datetime.timestamp()]
    filtered_comments_sorted = sorted(filtered_comments)
    if len(filtered_comments_sorted) <= 1:
        return 0
    first_comment_timestamp = datetime.fromtimestamp(filtered_comments_sorted[0])
    last_comment_timestamp = datetime.fromtimestamp(filtered_comments_sorted[-1])
    timedelta_between_first_last_comment = last_comment_timestamp - first_comment_timestamp
    minutes_between_first_last_comment = (timedelta_between_first_last_comment.seconds * 1.0) / 60
    return (len(comment_unix_timestamp_array) * 1.0) / minutes_between_first_last_comment

def get_stories_with_recent_comment_velocity(stories, recency_in_minutes):
    stories_with_recent_comment_velocity = {}
    for story_id, story in stories.items():
        recent_comment_velocity = get_comments_per_minute_from_recent_comments(story['comment_timestamps'], recency_in_minutes)
        story['recent_comments_per_minute'] = recent_comment_velocity
        stories_with_recent_comment_velocity[story_id] = story
    return stories_with_recent_comment_velocity

def convert_sorted_stories_to_display_list(sorted_stories):
    original_list = [sorted_story[1] for sorted_story in sorted_stories]
    display_list = []
    for story in original_list:
        display_story = story.copy()
        if 'comment_timestamps' in display_story:
            del display_story['comment_timestamps']
        display_list.append(display_story)
    return display_list

def get_recent(sorted_stories_display_list, minutes_to_consider_recent):
    return [sorted_story for sorted_story in sorted_stories_display_list \
            if (sorted_story['timedelta_since_post'].seconds / 60) < minutes_to_consider_recent]


###
###
###

## EDIT THIS TO CHANGE DEFINITION OF "RECENT COMMENT"
MINUTES_TO_CONSIDER_RECENT = 120


###
###
###

## SORT AND FORMAT STORIES BY RELEVANT METRICS

story_info_tracker = get_stories_with_recent_comment_velocity(story_info_tracker, MINUTES_TO_CONSIDER_RECENT)

num_comments_sorted_stories = sorted(story_info_tracker.items(), key=lambda x: x[1]['num_comments'], reverse=True)
comment_velocity_sorted_stories = sorted(story_info_tracker.items(), key=lambda x: x[1]['comments_per_minute'], reverse=True)
recent_comment_velocity_sorted_stories = sorted(story_info_tracker.items(), key=lambda x: x[1]['recent_comments_per_minute'], reverse=True)

num_comments_sorted_stories_display = convert_sorted_stories_to_display_list(num_comments_sorted_stories)
comment_velocity_sorted_stories_display = convert_sorted_stories_to_display_list(comment_velocity_sorted_stories)
recent_comment_velocity_sorted_stories_display = convert_sorted_stories_to_display_list(recent_comment_velocity_sorted_stories)

In [9]:
# get_recent(comment_velocity_sorted_stories_display, (3 * 60))
# get_recent(num_comments_sorted_stories_display, (3*60))

# recent_comment_velocity_sorted_stories_display

get_recent(comment_velocity_sorted_stories_display, (6 * 60))

[{'num_comments': 123,
  'title': 'Harry Browne’s 17 Golden Rules of Financial Safety (2013)',
  'timedelta_since_post': datetime.timedelta(seconds=9067, microseconds=35372),
  'time_since_post': '2h31m',
  'comments_per_minute': 0.8139406639461784,
  'url': 'https://news.ycombinator.com/item?id=37370858',
  'recent_comments_per_minute': 1.2772133526850509},
 {'num_comments': 110,
  'title': 'Sag-AFTRA votes unanimously to expand its strike to include the games industry',
  'timedelta_since_post': datetime.timedelta(seconds=14524, microseconds=348002),
  'time_since_post': '4h2m',
  'comments_per_minute': 0.4544202698980997,
  'url': 'https://news.ycombinator.com/item?id=37370170',
  'recent_comments_per_minute': 1.103759883814749},
 {'num_comments': 31,
  'title': 'I don’t want to grow my freelance design studio into an agency',
  'timedelta_since_post': datetime.timedelta(seconds=7250, microseconds=684314),
  'time_since_post': '2h0m',
  'comments_per_minute': 0.25655172413793104,
  

In [10]:
comment_velocity_df = pd.DataFrame.from_records(comment_velocity_sorted_stories_display)
recent_comment_velocity_df = pd.DataFrame.from_records(comment_velocity_sorted_stories_display)

In [ ]:
comment_velocity_df

In [ ]:
c = (alt.Chart(comment_velocity_df)
        .mark_area(opacity=0.3)
        .encode(
            x='timedelta_since_post',
            y='comments_per_minute',
            color='title',
    ))
st.altair_chart(c, use_container_width=True)

In [ ]:
st.dataframe(comment_velocity_df)

In [11]:
recent_comment_velocity_df.to_csv('recent_comment_velocity_df.csv')

In [19]:
time.max

datetime.time(23, 59, 59, 999999)